In [ ]:
import json
import pandas as pd
import json
import requests
from datetime import date

In [ ]:
def get_api_response(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        result = json.dumps(response.json(), sort_keys=True, indent=4)
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"
        
    return result

In [ ]:
stock=input()

In [ ]:
print("The Stock we will research is:" + stock)

In [ ]:
apikey= "l6rh6X3aJh8uy2817hEpF3bcFcHPzss37MGGujte"

url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stock}

headers = {
    'x-api-key': apikey
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

url2 = "https://yfapi.net/v11/finance/quoteSummary/" + str(stock) + "?lang=en&region=US&modules=defaultKeyStatistics"

response2 = requests.request("GET", url2, headers=headers, params=querystring)

url3 = "https://yfapi.net/ws/insights/v1/finance/insights?symbol=" + str(stock)

response3 = requests.request("GET", url3, headers=headers, params=querystring)

url4 = "https://yfapi.net/v11/finance/quoteSummary/" + str(stock) + "?lang=eng&region=US&modules=financialData"

response4= requests.request("GET", url4, headers=headers, params=querystring)


In [ ]:
stock_json = response.json() 

stock_json2 = response2.json()

stock_json3 = response3.json()

stock_json4 = response4.json()


In [ ]:
print(stock_json["quoteResponse"]["result"][0]['symbol'] + "; " +
     stock_json['quoteResponse']['result'][0]['longName'] +"; " + 
     str(stock_json['quoteResponse']['result'][0]['regularMarketPrice']) + '; ' +
     str(stock_json3['finance']['result']['instrumentInfo']['recommendation']["targetPrice"]) + "; " + 
     str(stock_json4["quoteSummary"]["result"][0]['financialData']['totalCash']['longFmt']) + '; '+ 
     stock_json2['quoteSummary']['result'][0]['defaultKeyStatistics']['profitMargins']['fmt'])

In [ ]:
df_stock = pd.DataFrame({
    "Ticker Name" : [stock_json["quoteResponse"]["result"][0]['symbol']],
    "Name of Company" : [stock_json['quoteResponse']['result'][0]['longName']],
    "Current Price" : [stock_json['quoteResponse']['result'][0]['regularMarketPrice']],
    "Target Mean Price" : [stock_json3['finance']['result']['instrumentInfo']['recommendation']["targetPrice"]],
    "Cash on Hand" : [stock_json4["quoteSummary"]["result"][0]['financialData']['totalCash']['longFmt']],
    "Profit Margins" : [stock_json2['quoteSummary']['result'][0]['defaultKeyStatistics']['profitMargins']['fmt']],
    "Date Data Was Pulled": [date.today()]
})

df_stock.to_json("API_JSON.json")